In [ ]:
%load_ext autoreload
%autoreload 2
# For reloading code such that code changes are taken into account

In [ ]:
project_id = 'test-project-221821'

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(os.environ['HOME'], 'gcp/secret/admin-service-account.json')
import sys
sys.path.append(os.path.join(os.environ['HOME'], 'gcp'))
import time

from pubsub import client as pubsub_client

In [ ]:
ps_client = pubsub_client.PubSub(project_id)

# Create a topic

First create a topic where we can publish messages to.

In [ ]:
topic_name = 'test_topic'

ps_client.create_topic(topic_name)

In [ ]:
print("The topic exists now: {}".format(ps_client.check_topic_existence(topic_name)))

# Create a subscription

This allows us to poll for messages. A subscription always belongs to one topic. A topic can have multiple subscriptions, but not the other way around.

In [ ]:
subscription_name = 'test_subscription'

ps_client.create_subscription(subscription_name, topic_name, ack_deadline_seconds=10)

# Publish messages on the topic

In [ ]:
data_to_publish = 'Hello pubsub!'
data_to_publish = data_to_publish.encode('utf-8')
attributes = {'value': '42'}


def publish_callback(message_future):
    """This callback let's you know whether a message was
    succesfully published.
    """
    if message_future.exception(timeout=20):
        print("Unable to publishd message.")
    else:
        print("Published message with id: {}".format(message_future.result()))

In [ ]:
ps_client.publish_message(topic_name, data_to_publish, publish_callback, **attributes)

# Pull messages from the subscription

In [ ]:
response_list = ps_client.pull_message(subscription_name)

In [ ]:
# In order to make sure the message doesn't get republished, we have to acknowledge the message within 10 seconds
if response_list:
    response = response_list[0]
    ack_id = response.ack_id
    message = response.message
    response_data = message.data.decode('utf-8')
    response_attributes = dict(message.attributes.items())
    print(response_data, response_attributes)
    ps_client.subscriber_client.acknowledge(ps_client.create_subscription_path(subscription_name), [ack_id])
else:
    print("No messages in queue anymore.")

# Streaming pull from a subscription

In [ ]:
def print_message(message):
    """Callback function for streaming pull.
    
    Args:
        message (google.cloud.pubsub_v1.subscriber.message.Message):
    """
    response_data = message.data.decode('utf-8')
    response_attributes = dict(message.attributes.items())
    print(response_data, response_attributes)
    message.ack()

In [ ]:
# Publish a number of messages on the topic
nr_messages_to_publish = 3
for i in range(nr_messages_to_publish):
    ps_client.publish_message(topic_name, data_to_publish, publish_callback, **attributes)

In [ ]:
# Start streaming pull for messages. Let's pick up the messages we just published.
streaming_pull_future = ps_client.subscribe(subscription_name, print_message)

In [ ]:
# Stop streaming pull
streaming_pull_future.cancel()

# Cleanup

In [ ]:
# This will not delete the subscription(s) that belong to the topic
ps_client.delete_topic(topic_name)

In [ ]:
ps_client.delete_subscription(subscription_name)